In [1]:
import numpy as np
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
print(torch.cuda.is_available())

/home/potate/miniconda3/envs/RL_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [2]:
from environment.connectFourEnv import ConnectFourEnv
from agent.agent import ConnectFourAgent
from agent.policy import VanillaPolicyModel

In [3]:
environment = ConnectFourEnv(
    win_reward = 1,
    blank_reward = 0
)

In [4]:
training_policy = VanillaPolicyModel()
training_agent = ConnectFourAgent(
    policy = training_policy,
    training = True,
    id_ = 1,
    lr = 1e-2
)

control_policy = VanillaPolicyModel()
control_agent = ConnectFourAgent(
    policy = control_policy,
    training = False,
    id_ = -1
)

In [5]:
NUM_EPOCHS = 10
BATCH_SIZE = 32

In [6]:
losses = {}
scores = {1 : 0, -1 : 0}
for e in tqdm(range(NUM_EPOCHS)):
    training_agent.no_grad()
    
    batch_losses = {}
    for b in range(BATCH_SIZE):
        is_over = False
        run_loss = None
        batch_loss = None
        training_agent.reset_reward()
        environment.reset_state()
        while not is_over:
            sampling_dist = control_agent.actions_distribution(environment.state)
            action = sampling_dist.sample()
            is_over, reward = control_agent.perform_action(action, environment)
            if is_over:
                training_agent.reward -= reward
                scores[-1] += 1
            else:
                # we generate the logits from the state
                sampling_dist = training_agent.actions_distribution(environment.state)
                action = sampling_dist.sample()

                is_over, reward = training_agent.perform_action(action, environment)
                training_agent.reward += reward
                if is_over:
                    training_agent.reward += reward
                    scores[1] += 1
            
            
            sampling_dist.log_prob(action)
            if run_loss is None:
                run_loss = - sampling_dist.log_prob(action)
            else:
                run_loss = run_loss - sampling_dist.log_prob(action)
                
            if is_over:
                print('--------------')
                print(training_agent.reward)
                print(run_loss.item())
                print('game is over')
                print('--------------')
            
        if batch_loss is None:
            batch_loss = run_loss * training_agent.reward / BATCH_SIZE
        else:
            batch_loss += run_loss * training_agent.reward / BATCH_SIZE
        batch_losses[b] = batch_loss.item()
        
    losses[e] = batch_losses
    batch_loss.backward()
    training_agent.optimizer.step()

  0%|                                                                                                                                       | 0/10 [00:00<?, ?it/s]

--------------
-1
22.814088821411133
game is over
--------------
--------------
2
25.045602798461914
game is over
--------------
--------------
-1
7.703569412231445
game is over
--------------
--------------
-1
22.559782028198242
game is over
--------------
--------------
-1
25.234100341796875
game is over
--------------
--------------
-1
21.34081268310547
game is over
--------------
--------------
2
19.381906509399414
game is over
--------------
--------------
2
9.6461820602417
game is over
--------------
--------------
-1
21.21198272705078
game is over
--------------
--------------
-1
24.863069534301758
game is over
--------------
--------------
2
15.494810104370117
game is over
--------------
--------------
-1
13.584799766540527
game is over
--------------
--------------
-1
26.457374572753906
game is over
--------------
--------------
-1
32.94169235229492
game is over
--------------
--------------
2
27.96120834350586
game is over
--------------
--------------
2
15.411783218383789
ga

 10%|████████████▋                                                                                                                  | 1/10 [00:00<00:02,  4.31it/s]

--------------
-1
13.78487491607666
game is over
--------------
--------------
-1
22.978591918945312
game is over
--------------
--------------
2
26.4988956451416
game is over
--------------
--------------
2
20.933929443359375
game is over
--------------
--------------
2
17.450624465942383
game is over
--------------
--------------
-1
35.02569580078125
game is over
--------------
--------------
2
13.481829643249512
game is over
--------------
--------------
2
17.420475006103516
game is over
--------------
--------------
-1
13.656877517700195
game is over
--------------
--------------
2
21.347076416015625
game is over
--------------
--------------
-1
27.357711791992188
game is over
--------------
--------------
2
32.6103401184082
game is over
--------------
--------------
-1
19.460742950439453
game is over
--------------
--------------
2
31.59471893310547
game is over
--------------
--------------
2
17.500356674194336
game is over
--------------
--------------
-1
19.401268005371094
game

 20%|█████████████████████████▍                                                                                                     | 2/10 [00:00<00:01,  4.18it/s]

...

--------------
-1
11.465048789978027
game is over
--------------
--------------
-1
15.798639297485352
game is over
--------------
--------------
-1
24.00169563293457
game is over
--------------
--------------
2
19.261415481567383
game is over
--------------
--------------
-1
7.376699924468994
game is over
--------------
--------------
-1
26.285724639892578
game is over
--------------
--------------
2
17.320968627929688
game is over
--------------
--------------
2
13.160337448120117
game is over
--------------
--------------
-1
23.505361557006836
game is over
--------------
--------------
2
7.746372699737549
game is over
--------------
--------------
-1
17.156492233276367
game is over
--------------
--------------
-1
27.250465393066406
game is over
--------------
--------------
2
15.788865089416504
game is over
--------------
--------------
-1
25.31046485900879
game is over
--------------
--------------
-1
9.351907730102539
game is over
--------------
--------------
2
18.767690658569336

 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎            | 9/10 [00:02<00:00,  4.32it/s]

--------------
-1
29.314468383789062
game is over
--------------
--------------
-1
13.660494804382324
game is over
--------------
--------------
-1
24.09992790222168
game is over
--------------
--------------
2
28.737653732299805
game is over
--------------
--------------
2
23.280094146728516
game is over
--------------
--------------
2
19.31449317932129
game is over
--------------
--------------
2
27.816150665283203
game is over
--------------
--------------
2
11.592287063598633
game is over
--------------
--------------
-1
11.226556777954102
game is over
--------------
--------------
2
20.484548568725586
game is over
--------------
--------------
2
29.11553192138672
game is over
--------------
--------------
-1
26.580219268798828
game is over
--------------
--------------
2
30.055355072021484
game is over
--------------
--------------
2
22.52889060974121
game is over
--------------
--------------
2
19.603866577148438
game is over
--------------
--------------
-1
25.919475555419922
ga

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.27it/s]

--------------
-1
21.448610305786133
game is over
--------------
--------------
-1
18.062681198120117
game is over
--------------
--------------
-1
16.042644500732422
game is over
--------------
--------------
-1
21.061790466308594
game is over
--------------
--------------
2
14.916817665100098
game is over
--------------
--------------
2
24.009418487548828
game is over
--------------
--------------
2
12.754755020141602
game is over
--------------
--------------
-1
9.498495101928711
game is over
--------------
--------------
-1
21.050262451171875
game is over
--------------
--------------
2
13.533740997314453
game is over
--------------
--------------
2
10.896244049072266
game is over
--------------
--------------
-1
25.419410705566406
game is over
--------------
--------------
2
18.943830490112305
game is over
--------------
--------------
-1
24.339569091796875
game is over
--------------


In [7]:
[np.mean(list(x.values())) for x in losses.values()]

[0.03640546603128314,
 0.2061882335692644,
 0.3715506438165903,
 0.2521201092749834,
 0.4102140935137868,
 0.007660208735615015,
 -0.010140019468963146,
 0.28608908131718636,
 0.34596845507621765,
 0.321663586422801]

In [8]:
scores

{1: 138, -1: 182}